In [1]:
import json
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
df_stations = pd.read_csv('stations_url.csv')
df_stations['url'] = 'https://www.belgiantrain.be' + df_stations['url']
df_stations

,name,url
0,Anvers,https://www.belgiantrain.be/fr/station-informa...
1,Anvers-Berchem,https://www.belgiantrain.be/fr/station-informa...
2,Anvers-Central,https://www.belgiantrain.be/fr/station-informa...
3,Anvers-Luchtbal,https://www.belgiantrain.be/fr/station-informa...
4,Anvers-Noorderdokken,https://www.belgiantrain.be/fr/station-informa...
...,...,...
561,Wervik,https://www.belgiantrain.be/fr/station-informa...
562,Wevelgem,https://www.belgiantrain.be/fr/station-informa...
563,Zedelgem,https://www.belgiantrain.be/fr/station-informa...
564,Zeebrugge-Dorp/Strand,https://www.belgiantrain.be/fr/station-informa...


In [3]:
scraper = cloudscraper.create_scraper()

In [4]:
def get_facilities(scraper, station_url):
    url = station_url
    r = scraper.get(url)
    if r.status_code == 200:
        html_doc = r.text
        soup = BeautifulSoup(html_doc)
        return [
            {
                'name': item['data-facility-name'],
                'value': item.get_text().strip().split('\r\n')[0],
            }
            for item in soup.find_all(
                'div',
                attrs={
                    'data-facility-name': True,
                }
            )
        ]
    else:
        print(f'Error with {url}: {r.status_code}, skipping')

In [5]:
stations = []
for i, station in enumerate(df_stations.to_dict(orient='records')):
    facilities = get_facilities(scraper, station['url'])
    stations.append({
        'station': station['name'],
        'facilities': facilities,
    })
    print(f'{i}. {station['name']} done')

0. Anvers done
1. Anvers-Berchem done
2. Anvers-Central done
3. Anvers-Luchtbal done
4. Anvers-Noorderdokken done
5. Anvers-Sud done
6. Balen done
7. Berlaar done
8. Boechout done
9. Booischot done
10. Boom done
11. Bornem done
12. Bouwel done
13. Duffel done
14. Ekeren done
15. Essen done
16. Geel done
17. Heide done
18. Heist-Op-Den-Berg done
19. Hemiksem done
20. Herentals done
21. Hoboken-Polder done
22. Hove done
23. Kalmthout done
24. Kapellen done
25. Kessel done
26. Kijkuit done
27. Kontich done
28. Lierre done
29. Malines done
30. Malines-Nekkerspoel done
31. Melkouwen done
32. Mol done
33. Mortsel done
34. Mortsel-Deurnesteenweg done
35. Mortsel-Liersesteenweg done
36. Mortsel-Oude-God done
37. Muizen done
38. Niel done
39. Nijlen done
40. Noorderkempen done
41. Olen done
42. Puurs done
43. Ruisbroek-Sauvegarde done
44. Schelle done
45. Sint-Katelijne-Waver done
46. Sint-Mariaburg done
47. Tielen done
48. Turnhout done
49. Wildert done
50. Willebroek done
51. Wolfstee done
52

In [6]:
with open('facilities.json', 'w') as f:
    json.dump(stations, f) 